In [ ]:
# Installa yt-dlp e ffmpeg in Google Colab
!pip install yt-dlp  # Installa la libreria yt-dlp per il download di video e audio
!apt-get install ffmpeg  # Installa ffmpeg, necessario per la manipolazione audio/video

import os  # Importa il modulo os per operazioni di sistema
import yt_dlp  # Importa la libreria yt_dlp per il download
from google.colab import files  # Importa il modulo files per scaricare file da Colab
import shutil  # Importa il modulo shutil per operazioni di file

# Crea una cartella per i file audio
output_path = 'download_music_py'  # Definisce il nome della cartella di output
if not os.path.exists(output_path):  # Controlla se la cartella esiste già
    os.makedirs(output_path)  # Se non esiste, crea la cartella
else:
    # Svuota la cartella se esiste
    for filename in os.listdir(output_path):  # Elenca tutti i file nella cartella
        file_path = os.path.join(output_path, filename)  # Costruisce il percorso completo del file
        if os.path.isfile(file_path):  # Controlla se l'elemento è un file
            os.remove(file_path)  # Elimina il file

def list_available_formats(video_url):
    """Restituisce i formati disponibili per il video."""
    with yt_dlp.YoutubeDL() as ydl:  # Crea un'istanza di YoutubeDL
        info_dict = ydl.extract_info(video_url, download=False)  # Estrae le informazioni del video senza scaricare
        formats = info_dict.get('formats', [])  # Ottiene la lista dei formati disponibili
        return formats  # Restituisce la lista dei formati

def download_youtube(video_url, format_id, is_audio):
    """Scarica il video o l'audio in base al formato selezionato."""
    ydl_opts = {  # Imposta le opzioni per il download
        'format': format_id,  # Specifica il formato da scaricare
        'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),  # Imposta il modello di output per il nome del file
        'postprocessors': [],  # Inizialmente non ci sono post-processori
    }

    # Se stiamo scaricando solo audio, impostiamo il postprocessor per convertire in MP3
    if is_audio:
        ydl_opts['postprocessors'] = [{
            'key': 'FFmpegExtractAudio',  # Specifica il post-processore per estrarre l'audio
            'preferredcodec': 'mp3',  # Imposta il codec audio preferito
            'preferredquality': '192',  # Imposta la qualità audio
        }]
    else:
        # Se stiamo scaricando video, possiamo anche garantire il formato MP4
        ydl_opts['postprocessors'] = [{
            'key': 'FFmpegVideoConvertor',  # Specifica il post-processore per convertire il video
            'preferedformat': 'mp4',  # Imposta il formato video preferito
        }]

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:  # Crea un'istanza di YoutubeDL con le opzioni specificate
        ydl.download([video_url])  # Scarica il video utilizzando l'URL fornito

# Richiesta all'utente di scegliere tra un video singolo e una playlist
download_type = input("Vuoi scaricare un video singolo (1) o una playlist (2)? ")

# Richiesta all'utente di inserire il link
if download_type == '1':
    video_url = input("Inserisci il link del video di YouTube: ")  # Chiede all'utente di inserire l'URL del video
    formats = list_available_formats(video_url)  # Ottiene i formati disponibili per il video

    # Mostra i formati disponibili
    print("Formati disponibili:")
    audio_formats = []  # Lista per i formati audio
    video_formats = []  # Lista per i formati video

    for i, f in enumerate(formats, start=1):  # Aggiunge un contatore per la selezione numerica
        if 'audio' in f['format']:  # Se il formato è audio
            audio_formats.append(f)  # Aggiungi il formato alla lista audio
            print(f"{i}: Audio - {f['format_id']} ({f.get('abr', 'N/A')} kbps, Note: {f.get('format_note', 'N/A')})")
        if 'video' in f['format']:  # Se il formato è video
            video_formats.append(f)  # Aggiungi il formato alla lista video
            print(f"{i + len(audio_formats)}: Video - {f['format_id']} ({f.get('abr', 'N/A')} kbps, Note: {f.get('format_note', 'N/A')})")

    # Seleziona il formato da scaricare
    media_type = input("Vuoi scaricare solo audio (1) o entrambi audio e video (2)? ")

    if media_type == '1':
        format_id = 'bestaudio'  # Scarica solo il miglior formato audio
        is_audio = True  # Imposta la variabile per indicare che si sta scaricando solo audio
    elif media_type == '2':
        format_id = 'bestvideo+bestaudio'  # Scarica sia video che audio
        is_audio = False  # Imposta la variabile per indicare che si sta scaricando anche video
    else:
        print("Opzione non valida. Chiudo il programma.")  # Messaggio di errore per opzione non valida
        exit()  # Esci dal programma

    # Verifica se il formato è valido e scarica il contenuto
    if format_id:
        print(f"Stai per scaricare il formato ID: {format_id}")  # Mostra l'ID del formato scelto
        download_youtube(video_url, format_id, is_audio)  # Scarica il contenuto
        print("Download completato.")  # Messaggio di completamento del download
    else:
        print("Formato non disponibile. Scegli un altro format_id.")  # Messaggio di errore per formato non disponibile

elif download_type == '2':
    playlist_url = input("Inserisci il link della playlist di YouTube: ")  # Chiede all'utente di inserire l'URL della playlist
    # Mostra i formati disponibili per il primo video della playlist
    formats = list_available_formats(playlist_url)  # Ottiene i formati disponibili per la playlist

    print("Formati disponibili per la playlist:")
    audio_formats = []  # Lista per i formati audio
    video_formats = []  # Lista per i formati video

    for i, f in enumerate(formats, start=1):  # Aggiunge un contatore per la selezione numerica
        if 'audio' in f['format']:  # Se il formato è audio
            audio_formats.append(f)  # Aggiungi il formato alla lista audio
            print(f"{i}: Audio - {f['format_id']} ({f.get('abr', 'N/A')} kbps, Note: {f.get('format_note', 'N/A')})")
        if 'video' in f['format']:  # Se il formato è video
            video_formats.append(f)  # Aggiungi il formato alla lista video
            print(f"{i + len(audio_formats)}: Video - {f['format_id']} ({f.get('abr', 'N/A')} kbps, Note: {f.get('format_note', 'N/A')})")

    # Seleziona il formato da scaricare per la playlist
    media_type = input("Vuoi scaricare solo audio (1) o entrambi audio e video (2)? ")

    if media_type == '1':
        format_id = 'bestaudio'  # Scarica solo il miglior formato audio
        is_audio = True  # Imposta la variabile per indicare che si sta scaricando solo audio
    elif media_type == '2':
        format_id = 'bestvideo+bestaudio'  # Scarica sia video che audio
        is_audio = False  # Imposta la variabile per indicare che si sta scaricando anche video
    else:
        print("Opzione non valida. Chiudo il programma.")  # Messaggio di errore per opzione non valida
        exit()  # Esci dal programma

    # Verifica se il formato è valido e scarica il contenuto
    if format_id:
        print(f"Stai per scaricare il formato ID: {format_id} per la playlist")  # Mostra l'ID del formato scelto
        download_youtube(playlist_url, format_id, is_audio)  # Scarica il contenuto
        print("Download playlist completato.")  # Messaggio di completamento del download
    else:
        print("Formato non disponibile. Scegli un altro format_id.")  # Messaggio di errore per formato non disponibile

else:
    print("Opzione non valida. Chiudo il programma.")  # Messaggio di errore per opzione non valida
    exit()  # Esci dal programma

# Comprimere la cartella in un file zip
shutil.make_archive(output_path, 'zip', output_path)  # Crea un file ZIP della cartella di output

# Scaricare il file zip
files.download(output_path + '.zip')  # Avvia il download del file ZIP
